#Importação

In [0]:
import pandas as pd
import pyspark
from pyspark.sql.functions import lit
from pyspark.sql.types import DateType, IntegerType, StringType, DoubleType
from datetime import datetime


#Bancos

In [0]:
## conferir 
lista_database = [''] 

# Geração de dados do relatório

In [0]:
#Declaração dos dataframes
df_detalhes = 0
df_detalhes_appended = pd.DataFrame()
data_atual = datetime.now().strftime("%d/%m/%Y")

#Inicio do Loop para ver os bancos de dados 
for db in lista_database_talend:
  df_metadata = spark.sql('SHOW TABLES IN  '+db)
  df_pandas_md  = df_metadata.select('database', 'tableName').toPandas()

# lista de Schemas e tabelas dentro do datafrane pandas
  for database,table in df_pandas_md.values:
    if not isinstance(df_detalhes, pyspark.sql.dataframe.DataFrame):
      try:  # Primeira passagem para cada database  
        df_detalhes = spark.sql(' DESCRIBE DETAIL '+database+'.'+table)
        print('1º Ok /', database+'.'+table, ' / criação')
      except:
         print('1º Error', database+'.'+table)
    else:
      try:  # Segunda passagem para cada database  
        df_detalhes = df_detalhes.union(spark.sql(' DESCRIBE DETAIL '+database+'.'+table))
        print('2º Ok /', database+'.'+table, ' / union')
      except:
        print('2º Error', database, table)
df_detalhes = df_detalhes.withColumn("dataExec", lit(data_atual).cast(StringType()))
df_detalhes = df_detalhes.toPandas()
df_detalhes_appended = df_detalhes_appended.append(df_detalhes)
  
df_final = spark.createDataFrame(df_detalhes_appended[['name','createdAt','lastModified', 'sizeInBytes', 'dataExec']].sort_values('lastModified', ascending=False)).toPandas()
print("Fim da carga.")
df_final.head()

1º Ok / dbsigadl.aluno  / criação
2º Ok / dbsigadl.aluno_documentacao  / union
2º Ok / dbsigadl.aproveitamento  / union
2º Ok / dbsigadl.banco  / union
2º Ok / dbsigadl.bi_st_periodo_letivo_ref  / union
2º Ok / dbsigadl.bloco  / union
2º Error dbsigadl candidato
2º Ok / dbsigadl.candidato_status_classificacao  / union
2º Ok / dbsigadl.candidatoxcodigo_promocional  / union
2º Ok / dbsigadl.cidade  / union
2º Ok / dbsigadl.codigo_promocional  / union
2º Ok / dbsigadl.complemento  / union
2º Ok / dbsigadl.complementoxparcela  / union
2º Ok / dbsigadl.componente_curricular  / union
2º Ok / dbsigadl.contato  / union
2º Ok / dbsigadl.cor_raca  / union
2º Ok / dbsigadl.curriculo  / union
2º Ok / dbsigadl.curriculo_composicao  / union
2º Ok / dbsigadl.curso  / union
2º Ok / dbsigadl.doc  / union
2º Ok / dbsigadl.documentacao  / union
2º Ok / dbsigadl.estado  / union
2º Ok / dbsigadl.evento  / union
2º Ok / dbsigadl.exames_avaliacoes_desempenho_discente  / union
2º Ok / dbsigadl.forma_ingresso 

,name,createdAt,lastModified,sizeInBytes,dataExec
0,spark_catalog.dbsigadl.bi_st_periodo_letivo_ref,2023-03-30 09:21:35.566,2023-03-30 09:22:05,39823,30/03/2023
1,spark_catalog.dbsigadl.turma_professor,2023-03-30 09:18:46.829,2023-03-30 09:19:15,2466471,30/03/2023
2,spark_catalog.dbsigadl.turmaxcomponente,2023-03-30 09:06:03.289,2023-03-30 09:06:36,9404427,30/03/2023
3,spark_catalog.dbsigadl.titulacao,2023-03-30 09:03:45.189,2023-03-30 09:04:20,45984,30/03/2023
4,spark_catalog.dbsigadl.funcionario_titulacao,2023-03-30 09:03:08.449,2023-03-30 09:03:54,305074,30/03/2023


#Criação de tabelas

In [0]:
#Cria o dataframe com o resultado final da geração de dados
#cria uma tabela temporária para fazer a comparação
df_final = spark.createDataFrame(df_final)
df_final.createOrReplaceTempView("ctrl_carga_temp")

##Comparação

In [0]:
%sql
--Cria uma tabela de falhas baseado nos dados que foram inseridos na tabela temporária que falharam
create or replace table stage.ctrl_carga_falhas as
  select org.* 
  from ctrl_carga_temp tmp
  inner join stage.ctrl_carga_falhas org
    on org.name = tmp.name 
  where org.lastModified >= tmp.lastModified
  and org.sizeInBytes >= tmp.sizeInBytes	

num_affected_rows,num_inserted_rows


In [0]:
#insere na tabela física os dados coletados pelo relatório
permanent_table_name = "stage.ctrl_carga_falhas"
df_final.write.mode('append').format("delta").saveAsTable(permanent_table_name)

In [0]:
%sql
--Exclui dados da tabela com a data de execução menos que o dia atual - 8
DELETE FROM stage.ctrl_carga_falhas
WHERE date_format(to_date(dataExec, 'dd/MM/yyyy'), 'yyyy-MM-dd') <= date_sub(current_date, 2)

num_affected_rows
0


In [0]:
%sql
--Exclui dados da tabela com a data de execução menos que o dia atual - 1
DELETE FROM stage.ctrl_carga
WHERE date_format(to_date(dataExec, 'dd/MM/yyyy'), 'yyyy-MM-dd') <= date_sub(current_date, 1)

num_affected_rows
0


##Extração do relatório

In [0]:
#Rekatório de falhas
df_falha = spark.read.table('stage.ctrl_carga_falhas')
df_falha.display()

extract,name,createdAt,lastModified,sizeInBytes,dataExec


In [0]:
#Relatório de cargas
df_table = spark.read.table('stage.ctrl_carga')
df_table.display()

extract,name,createdAt,lastModified,sizeInBytes,dataExec
null,spark_catalog.dbsigadl.bi_st_periodo_letivo_ref,2023-03-30T09:21:35.566+0000,2023-03-30T09:22:05.000+0000,39823,30/03/2023
null,spark_catalog.dbsigadl.turma_professor,2023-03-30T09:18:46.829+0000,2023-03-30T09:19:15.000+0000,2466471,30/03/2023
null,spark_catalog.dbsigadl.turmaxcomponente,2023-03-30T09:06:03.289+0000,2023-03-30T09:06:36.000+0000,9404427,30/03/2023
null,spark_catalog.dbsigadl.titulacao,2023-03-30T09:03:45.189+0000,2023-03-30T09:04:20.000+0000,45984,30/03/2023
null,spark_catalog.dbsigadl.funcionario_titulacao,2023-03-30T09:03:08.449+0000,2023-03-30T09:03:54.000+0000,305074,30/03/2023
null,spark_catalog.dbsigadl.funcionario,2023-03-30T08:37:16.231+0000,2023-03-30T08:37:45.000+0000,1331549,30/03/2023
null,spark_catalog.dbsigadl.instituicao_ensino_tipo,2023-03-30T08:34:14.962+0000,2023-03-30T08:34:40.000+0000,37098,30/03/2023
null,spark_catalog.dbsigadl.cor_raca,2023-03-30T08:19:08.451+0000,2023-03-30T08:21:51.000+0000,40715,30/03/2023
null,spark_catalog.dbsigadl.turno,2023-03-30T08:15:59.482+0000,2023-03-30T08:20:52.000+0000,54931,30/03/2023
null,spark_catalog.dbsigadl.nivel_ensino,2023-03-30T08:17:11.316+0000,2023-03-30T08:20:43.000+0000,50947,30/03/2023
